In [6]:
# Download the data https://www.iso-ne.com/static-assets/documents/100020/2025_smd_hourly.xlsx
import pandas as pd
data = pd.read_excel('2025_smd_hourly.xlsx', sheet_name='ISO NE CA')

data
# use 4 prev points of system load to predct the next one
data_shifted = []

for i in range(4, len(data)):
    data_shifted.append(
        data['System_Load'][i-4:i+1].values
)
    
data_shifted = pd.DataFrame(data_shifted, columns=['load_1_hour_MW', 'load_2_hour_MW', 'load_3_hour_MW', 'load_4_hour_MW', 'load_5_hour_MW'])
data_shifted = data_shifted / data_shifted.max() * 6400 # normalizing by nominal load of IEEEBUS39

data_train = data_shifted[:-100]
data_train.to_csv('data_new_engl_train.csv', index=False)

data_test = data_shifted[-100:]
data_test.to_csv('data_new_engl_test.csv', index=False)

In [7]:
# Xgboost
import xgboost as xgb

data = pd.read_csv('data_new_engl_train.csv')  # Load your dataset
X = data[[f'load_{i+1}_hour_MW' for i in range(4)]]
y = data['load_5_hour_MW']

#training
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model.fit(X, y)

#save
model.save_model('checkpoint.json')


In [9]:
# Torch
import torch

class SimpleNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
model = SimpleNN(input_size=4, hidden_size=10,output_size =1)

data = pd.read_csv('data_new_engl_train.csv')  # Load your dataset

X = data[[f'load_{i+1}_hour_MW' for i in range(4)]].values
y = data['load_5_hour_MW'].values

# training
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
num_epochs = 4
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        
        
#save       
traced_model = torch.jit.trace(model, torch.randn(1, 4))
traced_model.save("checkpoint.pth")

/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([8656])) that is different to the input size (torch.Size([8656, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
